In [1]:
"""
MFC Model-Predictive Control - Correct Dosage
==============================================
- Calculates the EXACT dose volume needed using CSTR physics
- Doses ONE substrate at a time
- Evaluates every 1 minute
- Priority: pH > TAN > EC
"""

import numpy as np

# =============================================================================
# SYSTEM PARAMETERS
# =============================================================================

V_INITIAL = 1000.0
DT = 1.0  # 1 minute

MIN_DOSE_ML = 0.5   # Minimum practical dose
MAX_DOSE_ML = 50.0  # Maximum single dose

# =============================================================================
# TARGETS (same as bang-bang for fair comparison)
# =============================================================================

TARGETS = {
    'pH':  {'min': 6.8, 'max': 7.2, 'setpoint': 7.0},
    'EC':  {'min': 5.0, 'max': 10.0, 'setpoint': 7.5},
    'TAN': {'min': 50.0, 'max': 200.0, 'setpoint': 125.0}
}

METAL_TARGETS = {
    'Fe': (1.0, 10.0), 'Zn': (0.1, 1.0), 'Cu': (0.01, 0.5),
    'Mn': (0.1, 1.0), 'Se': (0.01, 0.1), 'Ni': (0.05, 1.0), 'Co': (0.01, 0.3)
}

# =============================================================================
# DRIFT MODEL (same as bang-bang)
# =============================================================================

A_MATRIX = np.diag([-0.0001, -0.0001, -0.00003])
DRIFT = np.array([-0.0005, -0.001, -0.3])
METAL_DECAY_RATE = 0.0005

# =============================================================================
# SUBSTRATES (same as bang-bang)
# =============================================================================

SUBSTRATES = {
    'wine': {
        'pH': 4.0, 'EC': 0.02, 'TAN': 0.1,
        'Fe': 0.05, 'Zn': 0.01, 'Cu': 0.001, 'Mn': 0.02,
        'Se': 0.0, 'Ni': 0.0, 'Co': 0.0
    },
    'urine': {
        'pH': 8.5, 'EC': 6.25, 'TAN': 200.0,
        'Fe': 0.01, 'Zn': 0.05, 'Cu': 0.01, 'Mn': 0.005,
        'Se': 0.001, 'Ni': 0.002, 'Co': 0.001
    },
    'spirulina': {
        'pH': 9.0, 'EC': 1.14, 'TAN': 144.0,
        'Fe': 4.275, 'Zn': 0.300, 'Cu': 0.915, 'Mn': 0.285,
        'Se': 0.00105, 'Ni': 0.0, 'Co': 0.0
    }
}

# =============================================================================
# PHYSICS: CSTR MIXING EQUATION
# =============================================================================

def calculate_dose_to_target(current, target, substrate_val, volume):
    """
    Calculate exact dose volume to reach target using CSTR mixing.

    CSTR mixing: x_new = (x_current * V + x_substrate * V_dose) / (V + V_dose)

    Solving for V_dose:
        x_new * (V + V_dose) = x_current * V + x_substrate * V_dose
        x_new * V + x_new * V_dose = x_current * V + x_substrate * V_dose
        x_new * V - x_current * V = x_substrate * V_dose - x_new * V_dose
        V * (x_new - x_current) = V_dose * (x_substrate - x_new)
        V_dose = V * (x_target - x_current) / (x_substrate - x_target)
    """
    # Check if substrate can move us toward target
    if abs(substrate_val - current) < 1e-9:
        return 0  # Substrate same as current, can't help

    need_increase = (target > current)
    substrate_higher = (substrate_val > current)

    if need_increase != substrate_higher:
        return 0  # Wrong substrate - would move us away from target

    denom = substrate_val - target
    if abs(denom) < 1e-9:
        return MAX_DOSE_ML  # Substrate equals target, need infinite

    dose = volume * (target - current) / denom

    # Clamp to valid range
    if dose < MIN_DOSE_ML:
        return 0
    return min(dose, MAX_DOSE_ML)


def apply_dose(state, volume, substrate_name, dose_ml):
    """Apply CSTR mixing for a dose"""
    substrate = SUBSTRATES[substrate_name]
    new_vol = volume + dose_ml

    new_state = {}
    for key in state:
        if key in substrate:
            new_state[key] = (state[key] * volume + substrate[key] * dose_ml) / new_vol
        else:
            new_state[key] = state[key]

    return new_state, new_vol


def apply_drift(state, dt):
    """Apply autonomous dynamics (same as bang-bang)"""
    x = np.array([state['pH'], state['EC'], state['TAN']])
    dx_dt = A_MATRIX @ x + DRIFT
    x_new = x + dx_dt * dt

    decay = np.exp(-METAL_DECAY_RATE * dt)

    return {
        'pH': max(0, x_new[0]),
        'EC': max(0.01, x_new[1]),
        'TAN': max(0, x_new[2]),
        'Fe': state['Fe'] * decay,
        'Zn': state['Zn'] * decay,
        'Cu': state['Cu'] * decay,
        'Mn': state['Mn'] * decay,
        'Se': state['Se'] * decay,
        'Ni': state['Ni'] * decay,
        'Co': state['Co'] * decay,
    }

# =============================================================================
# MPC CONTROLLER - ONE SUBSTRATE AT A TIME
# =============================================================================

def mpc_control_decision(state, volume):
    """
    Physics-based control decision.

    Same priority as bang-bang:
    1. pH (highest) - bacterial survival
    2. TAN - nitrogen for growth
    3. EC - affects power output

    BUT: calculates exact dose volume to reach SETPOINT, not fixed 5mL

    Returns: (substrate_name or None, dose_ml, reason)
    """
    pH = state['pH']
    EC = state['EC']
    TAN = state['TAN']

    # === PRIORITY 1: pH control ===
    if pH > TARGETS['pH']['max']:
        # pH too high - need wine (acidic) to lower it
        dose = calculate_dose_to_target(pH, TARGETS['pH']['setpoint'],
                                        SUBSTRATES['wine']['pH'], volume)
        if dose >= MIN_DOSE_ML:
            return 'wine', dose, f"pH HIGH ({pH:.2f}>{TARGETS['pH']['max']}) -> dose to {TARGETS['pH']['setpoint']}"

    if pH < TARGETS['pH']['min']:
        # pH too low - need urine (basic) to raise it
        dose = calculate_dose_to_target(pH, TARGETS['pH']['setpoint'],
                                        SUBSTRATES['urine']['pH'], volume)
        if dose >= MIN_DOSE_ML:
            return 'urine', dose, f"pH LOW ({pH:.2f}<{TARGETS['pH']['min']}) -> dose to {TARGETS['pH']['setpoint']}"

    # === PRIORITY 2 & 3: TAN and EC combined logic (same as bang-bang) ===
    if TAN < TARGETS['TAN']['min'] and EC < TARGETS['EC']['min']:
        # Both low - urine provides both TAN and EC
        # Calculate dose for TAN (limiting factor usually)
        dose = calculate_dose_to_target(TAN, TARGETS['TAN']['setpoint'],
                                        SUBSTRATES['urine']['TAN'], volume)
        if dose >= MIN_DOSE_ML:
            return 'urine', dose, f"TAN+EC LOW (TAN={TAN:.1f}, EC={EC:.2f}) -> urine to TAN={TARGETS['TAN']['setpoint']}"

    if TAN < TARGETS['TAN']['min']:
        # TAN low, EC is OK - use spirulina (provides TAN without much EC change)
        dose = calculate_dose_to_target(TAN, TARGETS['TAN']['setpoint'],
                                        SUBSTRATES['spirulina']['TAN'], volume)
        if dose >= MIN_DOSE_ML:
            return 'spirulina', dose, f"TAN LOW ({TAN:.1f}<{TARGETS['TAN']['min']}) -> spirulina to {TARGETS['TAN']['setpoint']}"

    if EC < TARGETS['EC']['min']:
        # EC low, TAN is OK - use urine for EC
        dose = calculate_dose_to_target(EC, TARGETS['EC']['setpoint'],
                                        SUBSTRATES['urine']['EC'], volume)
        if dose >= MIN_DOSE_ML:
            return 'urine', dose, f"EC LOW ({EC:.2f}<{TARGETS['EC']['min']}) -> urine to EC={TARGETS['EC']['setpoint']}"

    # All in range
    return None, 0, "All parameters in range"

# =============================================================================
# SIMULATION
# =============================================================================

def run_simulation(duration_minutes=480, initial_state=None):
    if initial_state is None:
        initial_state = {
            'pH': 7.0, 'EC': 5.5, 'TAN': 80.0,
            'Fe': 1.0, 'Zn': 0.1, 'Cu': 0.05, 'Mn': 0.1,
            'Se': 0.01, 'Ni': 0.01, 'Co': 0.01
        }

    state = initial_state.copy()
    volume = V_INITIAL
    history = []

    total_volumes = {'wine': 0.0, 'urine': 0.0, 'spirulina': 0.0}
    dose_counts = {'wine': 0, 'urine': 0, 'spirulina': 0}
    dose_events = []

    print("=" * 140)
    print("MFC MODEL-PREDICTIVE CONTROL - PHYSICS-BASED DOSING")
    print("Calculates exact dose to reach setpoint | One substrate at a time | Every 1 minute")
    print("=" * 140)
    print(f"\nTargets: pH [{TARGETS['pH']['min']}-{TARGETS['pH']['max']}] setpoint={TARGETS['pH']['setpoint']}")
    print(f"         EC [{TARGETS['EC']['min']}-{TARGETS['EC']['max']}] setpoint={TARGETS['EC']['setpoint']} mS/cm")
    print(f"         TAN [{TARGETS['TAN']['min']}-{TARGETS['TAN']['max']}] setpoint={TARGETS['TAN']['setpoint']} mg-N/L")
    print(f"\nDose range: {MIN_DOSE_ML}-{MAX_DOSE_ML} mL")
    print("\n" + "=" * 140)
    print(f"\n{'Time':<6} {'pH':>6} {'EC':>8} {'TAN':>8} {'Vol':>8} {'Action':<60} {'Dose':>10}")
    print(f"{'(min)':<6} {'':>6} {'(mS/cm)':>8} {'(mg-N/L)':>8} {'(mL)':>8} {'':<60} {'(mL)':>10}")
    print("-" * 140)

    for t in range(int(duration_minutes) + 1):
        history.append({'time': t, 'volume': volume, **state.copy()})

        # MPC control decision
        action, dose_ml, reason = mpc_control_decision(state, volume)

        if action:
            action_str = f"DOSE {action.upper()}: {reason}"
            dose_str = f"{dose_ml:.1f}"
        else:
            action_str = reason
            dose_str = "---"

        # Print every line (every minute)
        print(f"{t:<6} {state['pH']:>6.2f} {state['EC']:>8.2f} {state['TAN']:>8.1f} "
              f"{volume:>8.0f} {action_str:<60} {dose_str:>10}")

        # Execute dose
        if action and dose_ml >= MIN_DOSE_ML:
            state, volume = apply_dose(state, volume, action, dose_ml)
            dose_counts[action] += 1
            total_volumes[action] += dose_ml
            dose_events.append({
                'time': t,
                'substrate': action,
                'volume_ml': dose_ml,
                'reason': reason
            })

        # Apply drift for next step
        if t < duration_minutes:
            state = apply_drift(state, DT)

    # Summary
    print("-" * 140)
    print("\n" + "=" * 140)
    print("SIMULATION SUMMARY")
    print("=" * 140)

    total_dosed = sum(total_volumes.values())
    print(f"\nDuration: {duration_minutes} minutes ({duration_minutes/60:.1f} hours)")
    print(f"Final Volume: {volume:.0f} mL")

    print(f"\nSubstrate Usage:")
    for sub in ['wine', 'urine', 'spirulina']:
        pct = (total_volumes[sub] / total_dosed * 100) if total_dosed > 0 else 0
        avg_dose = total_volumes[sub] / dose_counts[sub] if dose_counts[sub] > 0 else 0
        print(f"  {sub.capitalize():<12}: {total_volumes[sub]:>8.1f} mL ({pct:>5.1f}%) - {dose_counts[sub]:>3} doses (avg {avg_dose:.1f} mL/dose)")
    print(f"  {'Total':<12}: {total_dosed:>8.1f} mL - {sum(dose_counts.values())} doses")

    print(f"\nFinal State:")
    print(f"  pH:  {state['pH']:.2f} (target range: {TARGETS['pH']['min']}-{TARGETS['pH']['max']})")
    print(f"  EC:  {state['EC']:.2f} mS/cm (target range: {TARGETS['EC']['min']}-{TARGETS['EC']['max']})")
    print(f"  TAN: {state['TAN']:.1f} mg-N/L (target range: {TARGETS['TAN']['min']}-{TARGETS['TAN']['max']})")

    # Range compliance
    ph_ok = TARGETS['pH']['min'] <= state['pH'] <= TARGETS['pH']['max']
    ec_ok = TARGETS['EC']['min'] <= state['EC'] <= TARGETS['EC']['max']
    tan_ok = TARGETS['TAN']['min'] <= state['TAN'] <= TARGETS['TAN']['max']

    print(f"\nRange Compliance:")
    print(f"  pH:  {'✓ IN RANGE' if ph_ok else '✗ OUT OF RANGE'}")
    print(f"  EC:  {'✓ IN RANGE' if ec_ok else '✗ OUT OF RANGE'}")
    print(f"  TAN: {'✓ IN RANGE' if tan_ok else '✗ OUT OF RANGE'}")

    print(f"\nFinal Trace Metals:")
    for metal, (lo, hi) in METAL_TARGETS.items():
        val = state[metal]
        status = "OK" if lo <= val <= hi else ("LOW" if val < lo else "HIGH")
        print(f"  {metal}: {val:.4f} mg/L (target: {lo}-{hi}) {status}")

    # Comparison with bang-bang
    print(f"\n" + "=" * 140)
    print("COMPARISON: MPC vs BANG-BANG (fixed 5mL)")
    print("=" * 140)
    bb_doses = 333
    bb_volume = 1665
    print(f"  {'Metric':<25} {'MPC':>15} {'Bang-Bang':>15} {'Difference':>15}")
    print(f"  {'-'*25} {'-'*15} {'-'*15} {'-'*15}")
    print(f"  {'Total doses':<25} {sum(dose_counts.values()):>15} {bb_doses:>15} {sum(dose_counts.values())-bb_doses:>+15}")
    print(f"  {'Total volume (mL)':<25} {total_dosed:>15.1f} {bb_volume:>15} {total_dosed-bb_volume:>+15.1f}")
    if sum(dose_counts.values()) > 0:
        print(f"  {'Avg dose size (mL)':<25} {total_dosed/sum(dose_counts.values()):>15.1f} {5.0:>15.1f} {'':>15}")

    return history, dose_events


if __name__ == "__main__":
    history, events = run_simulation(duration_minutes=480)

MFC MODEL-PREDICTIVE CONTROL - PHYSICS-BASED DOSING
Calculates exact dose to reach setpoint | One substrate at a time | Every 1 minute

Targets: pH [6.8-7.2] setpoint=7.0
         EC [5.0-10.0] setpoint=7.5 mS/cm
         TAN [50.0-200.0] setpoint=125.0 mg-N/L

Dose range: 0.5-50.0 mL


Time       pH       EC      TAN      Vol Action                                                             Dose
(min)          (mS/cm) (mg-N/L)     (mL)                                                                    (mL)
--------------------------------------------------------------------------------------------------------------------------------------------
0        7.00     5.50     80.0     1000 All parameters in range                                             ---
1        7.00     5.50     79.7     1000 All parameters in range                                             ---
2        7.00     5.50     79.4     1000 All parameters in range                                             ---
3     